In [1]:
import pandas as pd
import numpy as np
import jalali

In [2]:
def extract_season(month):
    if month in [1, 2, 3]:
        return 1, "Spring", "بهار"
    elif month in [4, 5, 6]:
        return 2, "Summer", "تابستان"
    elif month in [7, 8, 9]:
        return 3,"Autumn", "پاییز"
    elif month in [10, 11, 12]:
        return 4, "Winter", "زمستان"
    else:
        return None

def extract_month_name(month):
    if month == 1:
        return "Farvardin", "فروردین"
    elif month == 2:
        return "Ordibehesht", "اردیبهشت"
    elif month == 3:
        return "Khordad", "خرداد"
    elif month == 4:
        return "Tir", "تیر"
    elif month == 5:
        return "Mordad", "مرداد"
    elif month == 6:
        return "Shahrivar", "شهریور"
    elif month == 7:
        return "Mehr", "مهر"
    elif month == 8:
        return "Aban", "آبان"
    elif month == 9:
        return "Azar", "آذر"
    elif month == 10:
        return "Dey", "دی"
    elif month == 11:
        return "Bahman", "بهمن"
    elif month == 12:
        return "Esfand", "اسفند"
    else:
        return None

def extract_wateryear(month, year):
    if month in [7, 8, 9, 10, 11, 12]:
        y_1 = year
        y_2 = year + 1
        return str(y_1) + "-" + str(y_2)[2:4]
    elif month in [1, 2, 3, 4, 5, 6]:
        y_1 = year - 1
        y_2 = year
        return str(y_1) + "-" + str(y_2)[2:4]
    else:
        return None

In [3]:
myDate = pd.DataFrame(
    data={
        "timestamp": pd.to_datetime(
            arg=pd.date_range(
                start="1940-01-01 00:00:00",
                end="2041-06-30 00:00:00",
                freq="30min"
            ).strftime("%Y-%m-%d %H:%M:%S"),
            format="%Y-%m-%d %H:%M:%S"
        ).to_list()
    }
)

myDate["date_gregorian"] = myDate["timestamp"].dt.date
myDate["date"] = myDate.apply(lambda column: jalali.Gregorian(column["date_gregorian"]).persian_string(), axis=1)
myDate[["year", "month", "day"]] = myDate["date"].str.split("-", 2, expand=True)
myDate[["year", "month", "day"]] = myDate[["year", "month", "day"]].apply(pd.to_numeric)
myDate["time"] = myDate["timestamp"].dt.time
myDate[["season", "season_english", "season_persian"]] = [extract_season(month) for month in myDate["month"]]
myDate[["month_english", "month_persian"]] = [extract_month_name(month) for month in myDate["month"]]
myDate["wateryear"] = [extract_wateryear(month, year) for month, year in zip(myDate["month"], myDate["year"])]
myDate["date_gregorian"] = pd.to_datetime(arg=myDate["date_gregorian"], format="%Y-%m-%d")

In [4]:
myDate.to_parquet('myDate.parquet.gzip',
                  compression='gzip')